In [5]:
from tqdm import tqdm
from collections import Counter
import random

In [6]:
FILE_NAME="clustering1.txt"

In [61]:
%%bash
FILE_NAME="clustering_big.txt"
LINK="https://d3c33hcgiwev3.cloudfront.net/_fe8d0202cd20a808db6a4d5d06be62f4_clustering_big.txt?Expires=1616457600&Signature=D1Za35jPlT4iil01J-4T~cb0e1YBVm9OkLYWQ2nXPMJnFrVTw6AHa4oQ51aQEeAw3XRFQbpC6gxCi1BZVvakGDUxjpDm9yxGR2T4ezRxkAcGYu2Nnb8EfeuYcn9S5RhlPsAVZXzWv8hujMd-~ULgpE2zGSz8sV3Y4soiqwwCsAY_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

wget -O $FILE_NAME $LINK 

In [62]:
%%bash
FILE_NAME="clustering1.txt"
LINK="https://d3c33hcgiwev3.cloudfront.net/_fe8d0202cd20a808db6a4d5d06be62f4_clustering1.txt?Expires=1616457600&Signature=elw6Wj3zWPq7H8s3qlm09vT~D9tWtV~uWW4ak3o6vi3OOIG1Pf-lk9Nd0gR~n2xzxMXG4bEZBB-Oh2tdvFueHcQt1lWCoaHuqbZzFkZsJM51cxnB7EgXRwp-Uia~hbM38NaeHiPIb0CM3PFZkJMAk~Hi8qeCROcZFcnDtsVNL6Y_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

wget -O $FILE_NAME $LINK 

In [63]:
! ls -lh

total 24712
-rw-r--r--  1 Alexander  staff    28K Mar 21 20:19 clustering.ipynb
-rw-r--r--  1 Alexander  staff   1.5M Mar 29  2016 clustering1.txt
-rw-r--r--  1 Alexander  staff   9.3M Mar 29  2016 clustering_big.txt


In [7]:
with open(FILE_NAME) as file:
    a = file.read().splitlines()

In [8]:
n_nodes = int(a[0])
k_clusters = 4

In [9]:
edges = [tuple(map(int, edge.split(' '))) for edge in a[1:]]

In [10]:
edges[:10]

[(1, 2, 6808),
 (1, 3, 5250),
 (1, 4, 74),
 (1, 5, 3659),
 (1, 6, 8931),
 (1, 7, 1273),
 (1, 8, 7545),
 (1, 9, 879),
 (1, 10, 7924),
 (1, 11, 7710)]

In [14]:
class UnionFind:
    def __init__(self,n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            # path compression
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def count(self):
        return self.n_clusters
        
    def union(self, x,y):
        root1 = self.find(x)
        root2 = self.find(y)
        
        if root1 == root2:
            return False
        
        # lazy union + union by rank
        if self.rank[root1] == self.rank[root2]:
            self.parent[root2] = root1
            self.rank[root1] += 1
        elif self.rank[root1] > self.rank[root2]:
            self.parent[root2] = root1
        else:
            self.parent[root1] = root2
        return True
            
    def connected(self, x, y):
        return self.find(x) == self.find(y)

In [17]:
from dataclasses import dataclass
@dataclass
class Edge:
    src: int
    dst: int
    weight: int


def clustering(num_nodes, k_clusters, edgelist):
    num_clusters = num_nodes
    edgelist.sort(key=lambda Edge : Edge.weight)
    U = UnionFind(num_nodes)
    spacing = 0
    for edge in tqdm(edgelist):
        united = U.union(edge.src, edge.dst)
        if united:
            num_clusters -= 1
            # k_clusters - 1 because I need to c
            # calculate next edge weigth as spacing
            if num_clusters == k_clusters - 1:
                spacing = edge.weight
                break
    return spacing

In [18]:
# subtract one so vertices start from 0
clustering(n_nodes, k_clusters, [Edge(x[0]-1, x[1]-1, x[2]) for x in edges])

  1%|          | 1217/124750 [00:00<00:00, 383650.35it/s]


97

# 2

In [79]:
vertices = ["".join(x.split(' ')) for x in open('clustering_big.txt', 'r').read().split('\n')[1:-1]]


In [81]:
vertices[:10]

['111000001101001111001111',
 '011001100101111110101101',
 '011100000001001011100101',
 '111010111011111010011000',
 '010011101011001111100100',
 '010010100001001110110101',
 '000110001101101010100000',
 '000010000111110111011000',
 '111001100100011111111001',
 '011110101101010000111111']

In [83]:
def invert(bit):
    if bit != '0' and bit != '1':
        raise ValueError
    return '1' if bit == '0' else '0'

def similar(v):
    out = []
    for i in range(len(v)):
        out.append(v[:i]+invert(v[i]) + v[i+1:])
        for j in range(i+1, len(v)):
            out.append(v[:i]+invert(v[i])+v[i+1:j]+invert(v[j])+v[j+1:])
    return out
 
heads = {}
for v in vertices:
    heads[v] = v
clusters = len(heads) 
#print clusters
for v in tqdm(vertices):
    v_head = heads[v]
    while heads[v_head] != v_head:
        v_head = heads[v_head]

    for friend in similar(v):
        if heads.get(friend):
            head = heads[friend]
            while heads[head] != head:
                head = heads[head]
            if v_head != head:
                heads[head] = v_head
                clusters -= 1
print(clusters)

100%|██████████| 200000/200000 [53:00<00:00, 62.88it/s] 

6118


In [ ]:
# 16508
# 6118